In [ ]:
import dijkstra3d
import numpy as np
import cc3d
import edt
import morphology
from pykdtree.kdtree import KDTree
import scipy

In [ ]:
"""import edt
axon_mask  # the binary image of the region of interest
zero_region_mask # X x Y x Z matrix the connected components of ID=0 touching the axon_mask
border_mask  # border voxels of axon_mask =1
border_indices # the indices into the image space that are border voxels of axon_mask
zero_region_indices # the indices into the cutout where zero_region_mask is one
dist_to_border # K long minimum euclidean distance of each voxel in zero_region_mask to border


total_mask = border_mask | axon_mask
total_mask_dist = edt.edt(total_mask, anisotropy=(8,8,40))
border_edt = total_mask_dist[border_mask]
mean_border_distance = np.mean(border_edt)
is_in_mylen_mask = dist_to_border<mean_border_distance
good_zero_indices=zero_region_indices[is_in_mylen_mask]
new_mask = np.zeros(zero_region_mask.shape)
new_mask[good_zero_indices]=1"""

In [ ]:
def get_mask_edt(mask):
    return edt.edt(mask, anisotropy=(8,8,40))

In [ ]:
def get_in_nm(pts, voxel_res):
    for i in range(len(pts)):
        for j in range(3):
            pts[i][j] = pts[i][j]*voxel_res[j]
    return pts

In [ ]:
def connectedCompLabeling(segmentation, connectivity, seg_id):
    labels_in = segmentation
    labels_out = cc3d.connected_components(labels_in, connectivity=connectivity)
    labels_out, N = cc3d.connected_components(labels_in, return_N=True)
    return labels_out, N

In [ ]:
def extract_connected_comps(label_id):
    extractedImg = labels_out * (labels_out == seg_id)
    process(extractedImg)
    return extractedImg
    

In [ ]:
def get_ids_in_mask(mask, id_space, exclude_list = (0)):
    id_list = id_space[mask]
    id_list = id_list[~np.isin(id_list,exclude_list)]
    vals, counts = np.unique(id_list, return_counts=True)
    return vals, counts

In [ ]:
def get_border_mask(mask_data):
    mask1 = morphology.binary_dilation(mask_data,iterations=1)
    mask2 = morphology.binary_dilation(mask1,iterations=2)
    border_mask = np.logical_xor(mask1,mask2)

In [ ]:
def generate_kdtree(data, leaf_size):
    return kd_tree = KDTree(data,leafsize=leaf_size)

In [ ]:
def query_tree(query_pts):
    nm_query_pts = get_in_nm(query_pts, [8, 8, 40])
    dist, idx = kd_tree.query(nm_query_pts, k=8)
    return dist

In [ ]:
def shortest_distance(border_mask, labels_out,kd_data):
    vals, counts = get_ids_in_mask(border_mask, labels_out)
    kd_tree = generate_kdtree(kd_data, 15)
    dist = np.empty(len(vals))
    for val in vals:
        query_pts = extract_connected_comps(val)
        dist.append(query_tree(query_pts))
        np.histogram(dist, bins=10)
    return vals, dist
